In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

from scipy.stats import ttest_ind
from sklearn.linear_model import LogisticRegression

* 여행자 보험 청구 여부를 예측하는 모델을 개발하고자 한다. 데이터셋은 다음과 같다.  

    travel_insurance.csv (구분자: 쉼표(,), 63,326 Rows, 12 Columns, UTF-8 Encoding, 결측치 존재(Null, N/A, Empty Cell), Quote 지정(쉼표로 분리된 문자열 존재)


In [4]:
dat=pd.read_csv('data/travel_insurance2.csv', na_values=['Null','N/A',''])
dat.dtypes

print('\n 변수목록: ', dat.columns)

print('\n 데이터 모양: ', dat.shape)


 변수목록:  Index(['NO', 'Agency', 'AgencyType', 'DistributionChannel', 'ProductName',
       'Claim', 'Duration', 'Destination', 'NetSales', 'Commision', 'Gender',
       'Age'],
      dtype='object')

 데이터 모양:  (63326, 12)


In [5]:
dat.Gender.value_counts()

M    9347
F    8872
Name: Gender, dtype: int64

* 분석 수행하기 전, 다음 전처리 단계들을 순서대로 수행하시오.  

    단계 1-1 ProductName, NetSales 변수를 제거하시오.

In [6]:
step1=dat.drop(columns=['ProductName', 'NetSales'])

    단계 1-2. 성별(Gender)에 결측치가 존재한다. 다음 규칙에 따라 성별의 결측치를 보정하시오.  
    
        If (NO=짝수) then (Gender=’M’) 
        Else if (NO=홀수) then (Gender=’F’)
        Else 성별(Gender)이 결측치가 아닌 경우 현재 값 유지

In [7]:
step2=step1.copy()

step2['Gender']=np.where(step2['Gender'].isna(),
                        np.where(step2['NO'] % 2 == 0, 'M', 'F' ),
                        step2['Gender'])

In [8]:
step2['Gender'].isna().sum() # 확인 0

0

In [9]:
basetable1=step2.copy()

1. (basetable1 이용) 40대(40<=Age<50)이고 여성(Gender=’F’)인 여행자가 두 번째로 많이 방문하는 여행지 국가(Destination)를 기술하시오  
(여행지 국가(Destination)는 대문자로 기술, 예시: ITALY)

    정답:	MALAYSIA

In [10]:
basetable1[basetable1.Age.between(40, 50)]

,NO,Agency,AgencyType,DistributionChannel,Claim,Duration,Destination,Commision,Gender,Age
4,5,CWT,Travel Agency,Online,No,79,ITALY,11.88,F,41
5,6,JZI,Airlines,Online,No,66,UNITED STATES,42.35,F,44
8,9,CWT,Travel Agency,Online,No,57,THAILAND,11.88,F,44
11,12,JZI,Airlines,Online,No,1,MALAYSIA,6.30,M,47
12,13,KML,Travel Agency,Online,No,53,NORWAY,49.40,F,48
...,...,...,...,...,...,...,...,...,...,...
63311,63312,SSI,Airlines,Online,No,96,SINGAPORE,1.66,M,48
63312,63313,SSI,Airlines,Online,No,61,SINGAPORE,1.84,F,48
63313,63314,SSI,Airlines,Online,No,60,SINGAPORE,0.74,M,48
63314,63315,JZI,Airlines,Online,No,59,THAILAND,7.70,F,44


In [11]:
Q1=basetable1[basetable1.Age.between(40, 50)][basetable1.Gender=='F']
Q1_out=Q1.Destination.value_counts()
Q1_out

SINGAPORE             1416
MALAYSIA               483
THAILAND               473
INDONESIA              375
AUSTRALIA              350
                      ... 
COLOMBIA                 1
GEORGIA                  1
NEPAL                    1
RUSSIAN FEDERATION       1
LATVIA                   1
Name: Destination, Length: 73, dtype: int64

In [12]:
Q1_ans=Q1_out.nlargest(2).index[-1]

print('Q1 정답: ', Q1_ans)  # MALAYSIA

Q1 정답:  MALAYSIA


2. (basetable1 이용) 유통 채널(DistributionChannel)에 따라 여행 기간(Duration)의 평균이 통계적으로 차이가 있는지 적절한 검정을 수행한 후, 검정 통계량(t-value)의 절댓값을 기술하시오.  

    - 등분산 가정하고 유의수준 0.05 하에서 양측검정  
    - 검정 통계량(t-value)의 절댓값은 소수점 둘째 자리까지만 기술(이후 자리는 버림, 예시: 0.12)  
    
    정답:	10.289600869594274


In [13]:
basetable1.DistributionChannel.value_counts()

Online     62219
Offline     1107
Name: DistributionChannel, dtype: int64

In [14]:
basetable1.Duration[basetable1.DistributionChannel=='Online']

2         65
3         60
4         79
5         66
6         47
        ... 
63321    111
63322     58
63323      2
63324      3
63325     22
Name: Duration, Length: 62219, dtype: int64

In [15]:
# from scipy.stats import ttest_ind

Q2=ttest_ind(basetable1.Duration[basetable1.DistributionChannel=='Online'],
            basetable1.Duration[basetable1.DistributionChannel=='Offline'],
            equal_var=True)

Q2_ans=Q2.statistic

print('Q2 정답: ', Q2_ans)  # -10.289600869594274

Q2 정답:  -10.289600869594274


3. (basetable1 이용) 마지막으로 청구(Claim) 여부를 예측하는 로지스틱 회귀 모형을 만들고자 합니다. 다음의 전처리를 수행한 후 분석을 수행하시오.   

    단계 1-3. 범주형 변수인 성별(Gender)와 유통 채널(DistributionChannel)에 대한 가변수(총 2개)를 추가하시오.  

        - 가변수화 Hint (Brightics Studio) : One Hot Encoder 사용, Drop Last=True

In [16]:
step3=pd.get_dummies(basetable1, 
                     columns=['Gender','DistributionChannel'],
                     drop_first=True)

    단계 1-4. 다음과 같이 Train과 Test 데이터셋을 분리하시오.  

        - Train DataSet: NO가 3의 배수가 아닌 데이터
        - Test DataSet: NO가 3의 배수인 데이터

In [17]:
train=step3[(step3.NO % 3) != 0]
test=step3[(step3.NO % 3) == 0]

In [18]:
train.head(3)

,NO,Agency,AgencyType,Claim,Duration,Destination,Commision,Age,Gender_M,DistributionChannel_Online
0,1,CBH,Travel Agency,No,186,MALAYSIA,9.57,81,0,0
1,2,CBH,Travel Agency,No,186,MALAYSIA,9.57,71,0,0
3,4,CWT,Travel Agency,No,60,AUSTRALIA,23.76,32,1,1


In [19]:
test.head(3)

,NO,Agency,AgencyType,Claim,Duration,Destination,Commision,Age,Gender_M,DistributionChannel_Online
2,3,CWT,Travel Agency,No,65,AUSTRALIA,29.70,32,0,1
5,6,JZI,Airlines,No,66,UNITED STATES,42.35,44,0,1
8,9,CWT,Travel Agency,No,57,THAILAND,11.88,44,0,1


    단계 1-5. Train DataSet으로 로지스틱 회귀분석 모델(Logistic Regression Model)을 학습하시오.  
    
    - 독립변수(총 5개): Duration, Commision, Age, 성별(Gender)과 유통 채널 
                        (DistributionChannel)의 가변수(단계 1-3. 추가)  
    - 종속변수: 청구 여부(Claim)  
    - 함수 가이드(Brightics Studio) 
        Inverse of Regularization=100,000, Seed=1234,
        Penalty: l2, Solver: newton-cg, 나머지: Default


In [20]:
train.columns

Index(['NO', 'Agency', 'AgencyType', 'Claim', 'Duration', 'Destination',
       'Commision', 'Age', 'Gender_M', 'DistributionChannel_Online'],
      dtype='object')

In [21]:
# from sklearn.linear_model import LogisticRegression

x_var1=['Duration', 'Commision', 'Age', 
        'Gender_M', 'DistributionChannel_Online']

Q3=LogisticRegression(C=100000, 
                      random_state=1234,penalty='l2', solver='newton-cg')

Q3.fit(train[x_var1], train.Claim)

LogisticRegression(C=100000, random_state=1234, solver='newton-cg')

단계 1-6. 단계 1-5.에서 학습한 모델을 단계 1-4.에서 분리한 ‘Test DataSet’에 적용하여 청구 확률 (Claim=Yes 일 확률)을 구한 후, Lift 변수를 추가하시오.  

        Lift = 청구 확률/0.015  

     (※ 0.015: 전체 데이터 셋에서 Claim=Yes인 비율, 반올림 적용)  

    데이터셋을 청구 확률 (Claim=Yes 일 확률) 순으로 (내림 차순, DESC ORDER) 정렬한 뒤, 상위 100등 안쪽(<=, 이하)에 포함되는 데이터들에 대한 Lift의 합계를 기술하시오.(Lift의 합계는 정수 부분만 기술(예시: 12)  
    
    - Hint(Brightics Studio) : rank() over (partition by A order by B desc/asc) 활용


In [22]:
Q3_out=pd.DataFrame(Q3.predict_proba(test[x_var1])[:,1], columns=['pred_P'])

Q3_out['Lift']=Q3_out['pred_P']/0.015

In [23]:
Q3_ans=Q3_out['Lift'].nlargest(100).sum()

print('Q3 정답: ', Q3_ans)  
# 1344.105337707629(brightics)
# 1344.105369804768(python)

Q3 정답:  1344.105369804768
